In [0]:
pip install OpenAI

In [0]:
%pip install -U mlflow
dbutils.library.restartPython()

In [0]:
import json, math, time, re
from typing import List, Dict, Any
from openai import OpenAI
import mlflow
mlflow.openai.autolog()

from pyspark.sql.functions import col, lit, coalesce, substring_index, rand

In [0]:
from delta.tables import DeltaTable

In [0]:
API_KEY = dbutils.secrets.get(scope="SPOTIFY", key="APIKEY_DEEPSEEK")
API_URL = "https://api.deepseek.com"
client = OpenAI(api_key=API_KEY, base_url=API_URL)

In [0]:
def classify_with_deepseek(items: List[Dict[str, Any]]) -> Dict[str, Any]:
    payload = json.dumps(items, ensure_ascii=False)
    response = client.chat.completions.create(
        model=MODEL,
        temperature=TEMPERATURE,
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": payload}
        ]
    )
    content = response.choices[0].message.content
    return json.loads(content)

In [0]:
def parsed_to_spark_df(parsed: Dict[str, Any]):
    data_list = parsed.get("data", [])
    
    if not data_list:
        raise ValueError("A API DeepSeek retornou uma resposta vazia ou sem a chave 'data'.")

    data_out: List[Dict[str, Any]] = []
    for obj in data_list:
        data_out.append({
            "master_metadata_track_name":        obj.get("master_metadata_track_name"),
            "master_metadata_album_artist_name": obj.get("master_metadata_album_artist_name"),
            "master_metadata_album_album_name":  obj.get("master_metadata_album_album_name"),
            "style":                             obj.get("Style", "unknown"),
        })

    return spark.createDataFrame(data_out)

In [0]:
df_historico = spark.table("silver.spotify_eng.streaming_history_2014_202509")
df_estilos = spark.table("gold.spotify.spotify_musicas")

In [0]:
# Identifica TODAS as músicas no histórico que não têm um estilo correspondente em df_estilos
df_sem_estilo_completo = (
    df_historico.alias("h")
    .join(
        df_estilos.alias("e"),
        [
            col("h.master_metadata_track_name") == col("e.master_metadata_track_name"),
            col("h.master_metadata_album_artist_name") == col("e.master_metadata_album_artist_name"),
            col("h.master_metadata_album_album_name") == col("e.master_metadata_album_album_name"),
        ],
        "left_anti" 
    )
    .select(
        "h.master_metadata_track_name",
        "h.master_metadata_album_artist_name",
        "h.master_metadata_album_album_name"
    )
    .distinct()
)

total_sem_estilo = df_sem_estilo_completo.count()

print(f"Total de músicas distintas sem estilo encontradas: {total_sem_estilo}")

if total_sem_estilo == 0:
    dbutils.notebook.exit("Nenhuma música nova para classificar. Job concluído com sucesso.")

df_para_enriquecer = df_sem_estilo_completo.orderBy(rand()).limit(50)

json_data = [row.asDict() for row in df_para_enriquecer.collect()]
print(f"\nSelecionado um lote de {len(json_data)} músicas para enviar à API.")

In [0]:
system_prompt = """
You are an expert in music genres. Return ONLY valid JSON.

INPUT: a JSON array of objects with fields:
- "master_metadata_track_name"
- "master_metadata_album_artist_name"
- "master_metadata_album_album_name"

REQUIREMENTS:
- Return a SINGLE JSON OBJECT with exactly one key: "data".
- "data" MUST be a JSON array with the same length and order as the input.
- Each object MUST preserve the original fields and add:
  "Style": the most appropriate single genre in English (e.g., "Brazilian Popular Music", "Samba", "Rock", "Hip Hop").
- No extra keys besides "data". No markdown/code fences. No text outside JSON.

EXAMPLE INPUT (array):
[
  {"master_metadata_track_name":"Cabeça Vazia","master_metadata_album_artist_name":"Djavan","master_metadata_album_album_name":"D"}
]

EXAMPLE OUTPUT (object):
{"data":[
  {"master_metadata_track_name":"Cabeça Vazia","master_metadata_album_artist_name":"Djavan","master_metadata_album_album_name":"D","Style":"Brazilian Popular Music"}
]}
""".strip()

TEMPERATURE = 0.0
MODEL = "deepseek-chat"

In [0]:
parsed_result = classify_with_deepseek(json_data)

In [0]:
df_novos_estilos = parsed_to_spark_df(parsed_result)

print("Estilos recebidos da API para o lote atual:")
display(df_novos_estilos)

In [0]:
updates = df_novos_estilos.alias("updates")
target_table = DeltaTable.forName(spark, "gold.spotify.spotify_musicas")

(
    target_table.alias("target")
    .merge(
        updates,
        "target.master_metadata_track_name = updates.master_metadata_track_name AND "
        "target.master_metadata_album_artist_name = updates.master_metadata_album_artist_name AND "
        "target.master_metadata_album_album_name = updates.master_metadata_album_album_name"
    )
    .whenNotMatchedInsertAll()
    .execute()
)
print("Tabela 'gold.spotify.spotify_musicas' atualizada com sucesso.")


In [0]:
df_estilos_atualizada = spark.table("gold.spotify.spotify_musicas")

df_historico_com_estilo = (
    df_historico.alias("h")
    .join(
        df_estilos_atualizada.alias("e"),
        [
            col("h.master_metadata_track_name") == col("e.master_metadata_track_name"),
            col("h.master_metadata_album_artist_name") == col("e.master_metadata_album_artist_name"),
            col("h.master_metadata_album_album_name") == col("e.master_metadata_album_album_name"),
        ],
        "left"
    )
    .select("h.*", "e.style")
)


df_historico_com_estilo.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("gold.spotify.streaming_history_com_estilos")

print("Tabela 'gold.spotify.streaming_history_com_estilos' criada com sucesso.")